In [1]:
import cv2
import numpy as np
import dlib
import os
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from skimage.feature import hog
from skimage import exposure

In [2]:
dir_train = r'.\preprocessed_dataset\Men\train'
dir_test = r'.\preprocessed_dataset\Men\test'
dir_model = r'.\model'


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dir_model + "\\" + "shape_predictor_68_face_landmarks.dat")

In [3]:
def extract_features(image):
    # Resize the image to 128x128
    image_resized = cv2.resize(image, (128, 128))
    # Compute HOG descriptors
    features, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(4, 4), block_norm="L2", visualize=True)
    return features, hog_image

In [4]:
def feature_engineering(path):
    labels = []
    hog_features = []
    shape_types = ['Heart', 'Oblong', 'Oval', 'Round', 'Square']

    for shape in shape_types:
        print(f'Loading {shape} images...')
        traindir = os.listdir(path + "\\" + shape)
        for imgName in traindir:
            print(f'Processing {imgName}...', end=" ")
            img_path = os.path.join(path, shape, imgName)
            img = cv2.imread(img_path)

            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Hog feature
            hog_feature , _ = extract_features(img)
            hog_features.append(hog_feature)
            labels.append(shape_types.index(shape))
        print("")
        
    labels = np.array(labels).reshape(len(labels), 1)
    hog_features = np.array(hog_features)

    data = np.hstack((hog_features, labels))
    return data

In [5]:
data_train = feature_engineering(dir_train)
data_test = feature_engineering(dir_test)

Loading Heart images...
Processing Heart (10).jpg... Processing Heart (102).jpg... Processing Heart (103).jpg... Processing Heart (104).jpg... Processing Heart (105).jpg... Processing Heart (106).jpg... Processing Heart (11).jpg... Processing Heart (112).jpg... Processing Heart (114).jpg... Processing Heart (116).jpg... Processing Heart (118).jpg... Processing Heart (121).jpg... Processing Heart (122).jpg... Processing Heart (125).jpg... Processing Heart (127).jpg... Processing Heart (129).jpg... Processing Heart (13).jpg... Processing Heart (131).jpg... Processing Heart (133).jpg... Processing Heart (134).jpg... Processing Heart (136).jpg... Processing Heart (137).jpg... Processing Heart (138).jpg... Processing Heart (14).jpg... Processing Heart (140).jpg... Processing Heart (142).jpg... Processing Heart (144).jpg... Processing Heart (145).jpg... Processing Heart (148).jpg... Processing Heart (149).jpg... Processing Heart (151).jpg... Processing Heart (152).jpg... Processing Heart (15

In [6]:
np.random.shuffle(data_train)
x_train = data_train[:, :-1]
y_train = data_train[:, -1]
np.random.shuffle(data_test)
x_test = data_test[:, :-1]
y_test = data_test[:, -1]

## ĐÂY LÀ PHẦN BUILD MODEL

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

para = {
    'C': [1, 2, 5],
    'gamma': [0.1, 0.01, 1, "scale"]
}
model = SVC(kernel = 'rbf', random_state = 42)
search = GridSearchCV(model, para)
search.fit(x_train[:300], y_train[:300])

GridSearchCV(estimator=SVC(random_state=42),
             param_grid={'C': [1, 2, 5], 'gamma': [0.1, 0.01, 1, 'scale']})

In [8]:
best_params = search.best_params_
best_score = search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'C': 2, 'gamma': 'scale'}
Best cross-validation score: 0.5


In [9]:
from sklearn.svm import SVC

model = SVC(kernel = 'rbf', C = 2, gamma="scale")
model.fit(x_train, y_train)

SVC(C=2)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[26  8 12  2  2]
 [ 4 27 11  4  4]
 [ 7  1 37  2  3]
 [ 1  3  6 36  4]
 [ 5  6  3  1 35]]


0.644

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.69      0.69      0.69       200
         1.0       0.71      0.73      0.72       198
         2.0       0.58      0.55      0.56       198
         3.0       0.65      0.68      0.67       197
         4.0       0.77      0.75      0.76       200

    accuracy                           0.68       993
   macro avg       0.68      0.68      0.68       993
weighted avg       0.68      0.68      0.68       993



### Save model after training (if training on tensorflow, please save model using this [link](https://www.geeksforgeeks.org/save-and-load-models-in-tensorflow/))

In [ ]:
%pip install joblib

In [ ]:
import pickle

# DON'T RUN IF YOU ALREADY HAVE THE MODEL FILES
file_name = './model/ml-model.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(model, file)